Remove archetype prediction redundancies among archetypes that overlap MPRA library. 

Use sliding window and median archetype length to select archetype w best score. 

In [1]:
import os, sys

sys.path.append(os.getcwd()) # append current working directory

import config_readwrite as crw  # custom script for reading config
import pandas as pd
import pybedtools as pbt

import chr_functions

# config

In [4]:
config, cfn = crw.read("/wynton/home/ahituv/fongsl/biomarin/bin/config.neuron.mpra.orig.ini")

# read
section = "design"


LIB = "lib2"

if LIB == "lib1":
    TILES = config[section]["full_bed"]
    PATH = config[section]["PATH"]
    PREFIX = f"unq_tiles.{LIB}"
    VIERSTRA = os.path.join(PATH, f"{PREFIX}.x.vierstra.archetypes.bed")
    CLEANED = os.path.join(PATH, f"{PREFIX}.x.vierstra.archetypes.cleaned.bed")
    CLEANED_TABLE = os.path.join(PATH, f"{PREFIX}.x.vierstra.archetypes.cleaned.table.bed")


    section = "tfbs"
    crw.check(config, section)

    config[section]["vierstra"] = VIERSTRA  # write the tiles
    config[section]["vierstra.cleaned"] = CLEANED  # write the tiles
    config[section]["vierstra.cleaned.table"] = CLEANED_TABLE # max TFBS motif score, no redundancies within 13 bp. Not strand specific

else:
    
    TILES = "/wynton/group/ahituv/fongsl/projects/biomarin/library_2/design/biomarin-lib2-hg38-final.bed.gz"
    PATH = config[section]["path2"]
    PREFIX = "biomarin-lib2-hg38-final"
    VIERSTRA = os.path.join(PATH, f"{PREFIX}.x.vierstra.archetypes.bed.gz")
    CLEANED = os.path.join(PATH, f"{PREFIX}.x.vierstra.archetypes.cleaned.bed")
    CLEANED_TABLE = os.path.join(PATH, f"{PREFIX}.x.vierstra.archetypes.cleaned.table.bed")



    section = "tfbs2"
    crw.check(config, section)

    config[section]["vierstra"] = VIERSTRA  # write the tiles
    config[section]["vierstra.cleaned"] = CLEANED  # write the tiles
    config[section]["vierstra.cleaned.table"] = CLEANED_TABLE # max TFBS motif score, no redundancies within 13 bp. Not strand specific

crw.write(config, cfn)

# run bedtools intersection

In [6]:
# intersect coordinates with bed tools using the -wao command. 
cmd = " ".join(["qsub", 
                "$HOME/vierstra_archetype.sh", 
                TILES
                ])
if os.path.exists(VIERSTRA) is False:
    
    os.system(cmd)

# tiles 

In [45]:
if "#chr" not in (list(pd.read_csv(VIERSTRA, sep='\t', nrows=3))):
    
    # column names
    names = ["#chr", 'start_tile', "end_tile", "#chr_arch", "start_arch", "end_arch", "group",
        'arch', "strand", "model", "num_models", "overlap"]

    # load dataframe
    df = pd.read_csv(VIERSTRA, sep='\t', header=None, names=names, low_memory=False)

    # sort and save
    df.sort_values(by=list(df.columns[:3])).drop_duplicates().to_csv(VIERSTRA, sep='\t', index=False)
else:
    print('already rearranged columns')
    df = pd.read_csv(VIERSTRA, sep='\t', low_memory=False)

In [53]:
tf = df.loc[df["overlap"] > 0].copy()

# compute motif length
tf["motif_len"] = tf["end_arch"] - tf["start_arch"]

# annotate fully overlapping motifs
tf["motif_overlap"] = False

# full overlap when tf motif len == overlap len
tf.loc[tf['motif_len'] == tf["overlap"],
        "motif_overlap"] = True

tf.groupby("motif_overlap")["#chr"].count()  #about 10% false overlaps

motif_overlap
False     596803
True     5864039
Name: #chr, dtype: int64

## sanity check - each group, model has one motif length 
(not multiple motif lengths for a model, group)

- check the length(s) of the motifs. If more than one motif length per motif, should print
  
              model [1, 2, etc.]
- If one motif length, print

              model [1]

In [44]:
for col in ["model", "group"]:
    a = tf.groupby(col)["motif_len"].unique().reset_index()
    print(col,a["motif_len"].apply(lambda x: len(str(x).split(","))).unique()) # count the length of the unique motifs per model)
    
    print(len(set(tf[col])))

model [1]
1615
group [1]
282


## prepare dataframe for sliding window

## sliding window function

In [54]:
def slideWindowCollapseGroup(df):
    print(df.shape)

    collection = {}  # collect results

    # cols to query
    cols = ["#chr_arch", "start_arch", "end_arch", "group", "strand", "arch"]

    # compute motif length
    df["len"] = df.end_arch-df.start_arch

    # compute median length
    median_len = df.len.describe()["50%"]

    # per chromosome, query starts
    for chr_ in set(df['#chr_arch']):
        print(chr_)
        test = df.loc[df['#chr_arch'] == chr_].copy()

        for start in test['start_arch'].unique():

            # test windows + median tfbs len range
            window = test.loc[test['start_arch'].between(
                start, int(start+median_len)), cols]

            # no group in window
            if window.shape[0] == 0:
                continue

            # if unique groups in window only
            elif len(set(window["group"])) == window.shape[0]:

                # add to collection
                collection[f'{chr_}:{start}'] = window

                continue

            # possibly non-unique
            else:
                # get max score among tfs
                # preserve strand information
                max_scores = window.groupby(["group", "strand"])[
                    'arch'].max().reset_index()

                # right join to keep only max_scores
                window = pd.merge(window, max_scores, how="right")

                collection[f'{chr_}:{start}'] = window[cols]

    # combine results
    cleaned = pd.concat(collection.values()).drop_duplicates()
    print("got rid of N redundacies=", df.shape[0]-cleaned.shape[0],
          "old:", df.shape,
          "new", cleaned.shape)

    return cleaned

## Sliding window to remove redundant TFBS archetype predictions

In [55]:
cleaned = slideWindowCollapseGroup(df)

(7481106, 12)
chrX
chr15
chr21
chr4
chr9
chr22
chr18
chr20
chrY
chr19
chr13
chr14
chr10
chr7
chr11
chr8
chr6
.
chr3
chr5
chr17
chr12
chr2
chr1
chr16
got rid of N redundacies= 6971688 old: (7481106, 13) new (509418, 6)


## write cleaned file

In [14]:
cleaned.head()

,#chr_arch,start_arch,end_arch,group,strand,arch
0,chr13,19864629,19864643,CTCF,+,8.1280
1,chr13,19864627,19864647,GC-tract,+,6.1290
2,chr13,19864641,19864661,GC-tract,-,8.0062
3,chr13,19864637,19864653,HEN1,-,4.5627
4,chr13,19864633,19864650,KLF/SP/2,+,7.9062


In [16]:
cleaned.shape, df.shape

((421937, 6), (490268, 9))

In [17]:
cleaned.to_csv(CLEANED, sep='\t', index=False)

# intersect tiles back into cleaned archetype dataset 

In [21]:
A = pbt.BedTool(TILES)
B = pbt.BedTool(CLEANED)

C = os.path.join(PATH, f"{PREFIX}.x.vierstra.archetypes.cleaned.re-intersect.lib1.bed")

A.intersect(B, wa=True, wb=True, output=C)

<BedTool(/wynton/group/ahituv/biomarin/library_2/Design/biomarin-lib2-hg38-final.x.vierstra.archetypes.cleaned.re-intersect.lib1.bed)>

# format and pivot table for archetype data

In [23]:
if LIB == "lib1":
    keepcols =[0,1,2,5,10,14,15,16]
    names = ["#chr", 'start_tile', "end_tile","cell_origin","enh_id", "TFArch", "TFStrand", "TFscore"]
else:
    keepcols =[0,1,2,3,7,8,9]
    names = ["#chr", 'start_tile', "end_tile","id", "TFArch", "TFStrand", "TFscore"]
df = pd.read_csv(C, sep='\t', header=None, 
                usecols =keepcols, names=names
                ).drop_duplicates()
print(df.shape)

df.head()

/scratch/fongsl/ipykernel_47534/854304351.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(C, sep='\t', header=None,


(4471990, 7)


,#chr,start_tile,end_tile,id,TFArch,TFStrand,TFscore
0,chr22,31898081,31898351,shuffle_64,GC-tract,+,9.8098
1,chr22,31898081,31898351,shuffle_64,ZNF354,-,7.3128
2,chr22,31898081,31898351,shuffle_64,PRDM4,-,2.1860
3,chr22,31898081,31898351,shuffle_64,LEF1,-,7.8040
4,chr22,31898081,31898351,shuffle_64,IRF/2,+,7.8743


## add tile coordinates back in

In [24]:
df = chr_functions.makeCoorAnnot(df, "#chr", 'start_tile', "end_tile", "tile")
df.head()

,#chr,start_tile,end_tile,id,TFArch,TFStrand,TFscore,tile.coor
0,chr22,31898081,31898351,shuffle_64,GC-tract,+,9.8098,chr22:31898081-31898351
1,chr22,31898081,31898351,shuffle_64,ZNF354,-,7.3128,chr22:31898081-31898351
2,chr22,31898081,31898351,shuffle_64,PRDM4,-,2.1860,chr22:31898081-31898351
3,chr22,31898081,31898351,shuffle_64,LEF1,-,7.8040,chr22:31898081-31898351
4,chr22,31898081,31898351,shuffle_64,IRF/2,+,7.8743,chr22:31898081-31898351


## groupby max score regardless of strand (creates redundancies. too hard.) and pivot

In [25]:
table = pd.pivot(df.groupby(["tile.coor", "TFArch"])["TFscore"].max().reset_index(), 
        index="tile.coor", 
         columns="TFArch", 
         values="TFscore"
        ).reset_index()

table.head()

TFArch,tile.coor,AHR,AIRE,AP1/1,AP1/2,ARI5A,ARI5B,BATF,BCL6/1,BCL6/2,...,ZNF652,ZNF667,ZNF680,ZNF708,ZNF713,ZNF768,ZNF784,ZNF85,ZSCAN3,ZSCAN4
0,chr10:101031026-101031296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.4566,...,NaN,8.4637,NaN,7.1276,NaN,8.8228,NaN,NaN,7.1462,NaN
1,chr10:101031046-101031316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8.4637,NaN,7.1276,NaN,8.8228,NaN,NaN,7.1462,NaN
2,chr10:101031066-101031336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8.4637,NaN,7.1276,NaN,4.3834,NaN,NaN,NaN,NaN
3,chr10:101031086-101031356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8.4637,NaN,7.1276,NaN,4.3834,NaN,NaN,NaN,NaN
4,chr10:101031106-101031376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8.4637,NaN,7.1276,NaN,4.3834,NaN,NaN,NaN,NaN


## write table

In [26]:
table.drop_duplicates().to_csv(CLEANED_TABLE, sep='\t', index=False)

In [28]:
table.drop_duplicates().shape

(34913, 283)

In [27]:
CLEANED_TABLE

'/wynton/group/ahituv/biomarin/library_2/Design/biomarin-lib2-hg38-final.x.vierstra.archetypes.cleaned.table.bed'

In [ ]:
tabl